## 2. Sentiment Analysis

After downloading the data, next step is to clean the tweets and run sentiment analysis using VADER function and prepare the final dataset.

Kindly note that the Twitter data was downloaded in a batch of 7 days so duplicate notebooks were created to clean and perform sentiment analysis for both Bitcoin and Ethereum. After all the sentiment scores were calculated, everything was collated in one file separately for both currencies. 

**Output of this script is the original tweets data with sentiment scores for one batch of a particular currency (BITCOIN in this case)**

In [1]:
import pandas as pd
import numpy as np
import string
# !pip install --user -U nltk # if you don't have it
import nltk 
#nltk.download() # if you don't have it
from nltk.corpus import stopwords
import nltk.tokenize 
from nltk.stem import WordNetLemmatizer

#word count
from collections import Counter

#word clouds
import heapq
#from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import collections
import matplotlib.cm as cm
from matplotlib import rcParams
from tqdm import tqdm

#sentiment analysis
from textblob import TextBlob

#text preprocessing
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saggarwal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saggarwal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Merge Tweet Datasets

Merge all the datasets with tweet and user information together.

In [2]:
#read the tweet data 
data = pd.read_csv('tweet_data_1.csv')
data2=pd.read_csv('tweet_data_2.csv')
data3=pd.read_csv('tweet_data_3.csv')
data= data.append(data2, ignore_index = True)
data= data.append(data3, ignore_index = True)
data.head()

,author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,referenced_tweets_id,referenced_tweets_type,media
0,2753291192,2021-07-01 18:47:32+00:00,,1410671428450541568,en,0,0,0,110,Twitter for Android,RT @OfficialTravlad: 🔥 SafeRiseV2 🔥 \n\n🦅 Risi...,1410661961260572673,retweeted,NaN
1,150624586,2021-07-01 18:47:31+00:00,,1410671426495991812,en,1,0,0,0,Twitter for Android,Holly shit. I just found Mute functionality in...,0,NaN,NaN
2,1400283959041224706,2021-07-01 18:47:31+00:00,,1410671426265174025,en,0,0,0,8687,Twitter for Android,RT @MiniDOGEToken: MiniDOGE is about to take o...,1409645719842000902,retweeted,NaN
3,952526506051371008,2021-07-01 18:47:31+00:00,,1410671425401372673,en,0,0,0,2,Twitter for Android,RT @Bufi79921560: @_hamster_coin @cz_binance @...,1410638396666621957,retweeted,NaN
4,1371937887109980161,2021-07-01 18:47:29+00:00,,1410671418652643334,en,0,0,0,194,Twitter for Android,RT @easyeight08: 50k NEXT WEEK OR I DELETE MY ...,1410628035150241799,retweeted,NaN


In [3]:
#read the user data
data_user = pd.read_csv('data_user_1.csv')
data_user2=pd.read_csv('data_user_2.csv')
data_user3=pd.read_csv('data_user_3.csv')
data_user= data_user.append(data_user2, ignore_index = True)
data_user= data_user.append(data_user3, ignore_index = True)
data_user.head()

,author_id,created_at,location,verified,username,followers_count,following_count,listed_count,tweet_count,name,description
0,2753291192,2014-08-21 22:56:18+00:00,"Украина ,г.Винница",False,Igor771008,76,3698,0,18676,Игорь Б,I live in Ukraine\nBEP20 BSC: 0x708283ce129...
1,150624586,2010-06-01 11:57:06+00:00,Argentina,False,ashmanukyan,148,270,0,1337,Ash XRP ♻️ 🇦🇷 🇦🇲,Software Developer. crypto enthusiast.\nXRP AD...
2,1400283959041224706,2021-06-03 02:51:59+00:00,NaN,False,atmadimas666,2,136,0,222,Dimdim,NaN
3,952526506051371008,2018-01-14 13:03:10+00:00,"İstanbul, Türkiye",False,Mustafacapoglu1,118,416,1,4197,Mustafacapoglu🇹🇷🇹🇷,NaN
4,1371937887109980161,2021-03-16 21:34:26+00:00,NaN,False,IqrarHu82069920,5,150,0,39,Iqrar Hussain,Pharmacist 😍


In [4]:
#read the media data
data_media=pd.read_csv('media_data_ref_1.csv')
data_media2=pd.read_csv('media_data_ref_2.csv')
data_media3=pd.read_csv('media_data_ref_3.csv')
data_media= data_media.append(data_media2, ignore_index = True)
data_media= data_media.append(data_media3, ignore_index = True)
data_media.head()


,media_key,media_type,media_url,width,height,duration_ms,view_count,preview_image_url
0,3_1410671402227748870,photo,https://pbs.twimg.com/media/E5O2KpCWUAYpX5u.jpg,1080,1727,0,0,NaN
1,3_1410671382304808963,photo,https://pbs.twimg.com/media/E5O2Je0WYAMy7Ut.jpg,540,620,0,0,NaN
2,3_1410572184175661057,photo,https://pbs.twimg.com/media/E5Nb7Y_WUAE7cTI.jpg,2528,1630,0,0,NaN
3,3_1410671144403939335,photo,https://pbs.twimg.com/media/E5O17okXMAcIIp6.jpg,1600,900,0,0,NaN
4,3_1410618943539081218,photo,https://pbs.twimg.com/media/E5OGdJMVcAIVeXp.jpg,950,600,0,0,NaN


In [5]:
#read the tweet references data
data_tweet_ref=pd.read_csv('tweet_data_ref_1.csv')
data_tweet_ref2=pd.read_csv('tweet_data_ref_2.csv')
data_tweet_ref3=pd.read_csv('tweet_data_ref_3.csv')
data_tweet_ref= data_tweet_ref.append(data_tweet_ref2, ignore_index = True)
data_tweet_ref= data_tweet_ref.append(data_tweet_ref3, ignore_index = True)
data_tweet_ref.head()

,author_id,created_at,reply_settings,tweet_id,lang,like_count,quote_count,reply_count,retweet_count,source,"text,conversation_id",media
0,2021-07-01 18:09:55+00:00,everyone,1410661961260572673,en,473,0,53,110,Twitter for iPhone,🔥 SafeRiseV2 🔥 \n\n🦅 Rising from the bot-litte...,1410661961260572673,3_1410661955187130369
1,2021-06-28 22:51:44+00:00,everyone,1409645719842000902,en,7069,1908,1305,8687,Twitter for iPhone,MiniDOGE is about to take over the crypto spac...,1409645719842000902,NaN
2,2021-07-01 16:36:16+00:00,everyone,1410638396666621957,en,2,0,0,2,Twitter for Android,@_hamster_coin @cz_binance @Hotbit_news @jack ...,1410622467425583115,NaN
3,2021-07-01 15:55:06+00:00,everyone,1410628035150241799,en,2577,48,609,194,Twitter for iPhone,50k NEXT WEEK OR I DELETE MY TWITTER! $BTC,1410628035150241799,NaN
4,2021-07-01 11:36:19+00:00,everyone,1410562910833754114,en,1564,14,237,299,Twitter Web App,"Next bull run ❤️:\n#BTC : 100,000$\n#BNB : 2,0...",1410562910833754114,NaN


### Tweet Statistics

These Stats are for Bitcoin between 1 July - 15 july 2021

In [6]:
print("There are " + str(data.tweet.nunique()) + " unique tweets. \nThere are " \
      + str(data['tweet'].count()) + " overall tweets.")

There are 683273 unique tweets. 
There are 1727221 overall tweets.


In [7]:
print("There are " + str(data_user.author_id.nunique()) + " unique authors in user table. \nThere are " \
      + str(data['author id'].nunique()) + " unique authors in tweet table.")

There are 400441 unique authors in user table. 
There are 381361 unique authors in tweet table.


In [8]:
print("There are " + str(data_tweet_ref['text,conversation_id'].nunique()) + " unique conversations in tweet ref table. \nThere are " \
      + str(data['referenced_tweets_id'].nunique()) + " referenced tweets in tweet table.")

There are 159665 unique conversations in tweet ref table. 
There are 197987 referenced tweets in tweet table.


In [9]:
print("There are " + str(data_media['media_key'].nunique()) + " unique media in media table. \nThere are " \
      + str(data['media'].nunique()) + " media in tweet table.")

There are 176945 unique media in media table. 
There are 157924 media in tweet table.


### Data Manipulation

User data is de-duplicated as user data gets saved in the original csv file everytime tweet posted by the user is downloaded

In [11]:
#drop duplicate data from user data
data_user = data_user.sort_values(by=['author_id','created_at'],ascending=False )
data_user_ddup=data_user.drop_duplicates(subset=['author_id'],keep='first')
data_user_ddup.shape

(400441, 11)

In [12]:
merge = pd.merge(data,data_user_ddup, left_on='author id', right_on='author_id',how='left')

In [13]:
merge.shape

(1727221, 26)

In [58]:
merge.head(50)

,author id,created_at_x,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,...,name,description,media_key,media_type,media_url,width,height,duration_ms,view_count,preview_image_url
0,2753291192,2021-07-01 18:47:32+00:00,,1410671428450541568,en,0,0,0,110,Twitter for Android,...,Игорь Б,I live in Ukraine\nBEP20 BSC: 0x708283ce129...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,150624586,2021-07-01 18:47:31+00:00,,1410671426495991812,en,1,0,0,0,Twitter for Android,...,Ash XRP ♻️ 🇦🇷 🇦🇲,Software Developer. crypto enthusiast.\nXRP AD...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1400283959041224706,2021-07-01 18:47:31+00:00,,1410671426265174025,en,0,0,0,8687,Twitter for Android,...,Dimdim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,952526506051371008,2021-07-01 18:47:31+00:00,,1410671425401372673,en,0,0,0,2,Twitter for Android,...,Mustafacapoglu🇹🇷🇹🇷,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1371937887109980161,2021-07-01 18:47:29+00:00,,1410671418652643334,en,0,0,0,194,Twitter for Android,...,Iqrar Hussain,Pharmacist 😍,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3628204632,2021-07-01 18:47:29+00:00,,1410671417750790148,en,0,0,0,299,Twitter Web App,...,NAF Crypto,Digital Marketer | Crypto Enthusiasts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,712941404175998976,2021-07-01 18:47:29+00:00,,1410671417729765376,en,0,0,0,545,Twitter for Android,...,Nicolas,Feeling Lucky,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2163163850,2021-07-01 18:47:29+00:00,,1410671417591468037,en,0,0,0,5977,Twitter for Android,...,D🌼🍭,tengo dos pollitos 🐥🐣 \n🧁👩🏼‍🍳 Aprendiz 🧐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,436410772,2021-07-01 18:47:29+00:00,,1410671416341532677,en,1,0,1,0,Twitter for Android,...,Febri Kustiono,94 line | kau bisa menjadi apapun dalam hidup ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1147769435618287617,2021-07-01 18:47:28+00:00,,1410671412499468291,pt,0,0,0,1,Twitter,...,Melon Husk,#dogearmy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#merge the media data with user data
merge=pd.merge(merge,data_media,left_on='media', right_on='media_key',how='left')

In [15]:
merge1 = merge.drop_duplicates()
merge1.shape

(1740526, 34)

In [16]:
merge1.columns

Index(['author id', 'created_at_x', 'geo', 'id', 'lang', 'like_count',
       'quote_count', 'reply_count', 'retweet_count', 'source', 'tweet',
       'referenced_tweets_id', 'referenced_tweets_type', 'media', 'unique_id',
       'author_id', 'created_at_y', 'location', 'verified', 'username',
       'followers_count', 'following_count', 'listed_count', 'tweet_count',
       'name', 'description', 'media_key', 'media_type', 'media_url', 'width',
       'height', 'duration_ms', 'view_count', 'preview_image_url'],
      dtype='object')

### Data cleanup & Sentiment Analysis

To run the sentiment analysis, unique tweets were extracted to reduce the computation time and later merged with the original file to keep the retweets as part of the analysis

In [10]:
### unique identifier for each tweet
data['unique_id'] = data.groupby(['tweet']).ngroup()

In [43]:
data.sort_values(by=['unique_id']).tail()

,author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,referenced_tweets_id,referenced_tweets_type,media,unique_id
52172,960340187510460419,2021-07-01 10:57:36+00:00,,1410553168623198214,en,6,1,0,0,Twitter Web App,"🪶 #BTC - 👀 the chart &amp; so far so good, fol...",1410287918460162048,quoted,3_1410552724983271425,683268
1413268,960340187510460419,2021-07-10 14:27:14+00:00,,1413867412152332288,en,13,1,2,5,Twitter Web App,"🪶 #BTC - 👀 the chart, so far following the pla...",0,NaN,3_1413865740210515973,683269
788094,960340187510460419,2021-07-02 01:50:28+00:00,,1410777863444389889,en,11,0,2,1,Twitter Web App,🪶 #BTC funding rate is negative in most exchan...,0,NaN,3_1410777277638533123,683270
1438543,1377965422050484232,2021-07-10 09:31:12+00:00,,1413792914308845572,ar,3,0,0,3,Twitter for iPhone,🪶 مشكله معضم المتداولين هي بإدارة المخاطر \nيم...,0,NaN,NaN,683271
22068,1159813805951635458,2021-07-01 15:10:18+00:00,,1410616761024536584,nl,1,0,1,0,Twitter Web App,"🫐 Op een dag dat de handel weer lager is, is h...",0,NaN,3_1410615691544121358,683272


In [17]:
data_dd = data.drop_duplicates(subset=['unique_id', 'tweet'])
data_dd.shape

(683273, 15)

In [11]:
data_dd = data_dd[['unique_id','tweet']]

In [12]:
data_dd.head()

,unique_id,tweet
0,480075,RT @OfficialTravlad: 🔥 SafeRiseV2 🔥 \n\n🦅 Risi...
1,360524,Holly shit. I just found Mute functionality in...
2,476916,RT @MiniDOGEToken: MiniDOGE is about to take o...
3,446465,RT @Bufi79921560: @_hamster_coin @cz_binance @...
4,512353,RT @easyeight08: 50k NEXT WEEK OR I DELETE MY ...


Various techniques were tested to clean twitter data and finally four steps were performed -

1. remove twitter Return handles (RT @xxx:)
2. remove twitter handles (@xxx)
3. remove URL links (httpxxx)
4. remove special characters, numbers, punctuations (except for #)

Emoticons and rest of the hashtags, and capitals are kept as VADER takes these into consideration while calculating the sentiment score.


In [13]:
# Handling Abbreviation
import re
def decontracted(phrase):
    phrase = re.sub(r"that\'s", "that is", phrase)   
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

# decontracte words
data_dd['processed_tweet_new'] = data_dd.tweet.apply(lambda x: decontracted(x).lower())

In [14]:
data_dd.head()

,unique_id,tweet,processed_tweet_new
0,480075,RT @OfficialTravlad: 🔥 SafeRiseV2 🔥 \n\n🦅 Risi...,rt @officialtravlad: 🔥 saferisev2 🔥 \n\n🦅 risi...
1,360524,Holly shit. I just found Mute functionality in...,holly shit. i just found mute functionality in...
2,476916,RT @MiniDOGEToken: MiniDOGE is about to take o...,rt @minidogetoken: minidoge is about to take o...
3,446465,RT @Bufi79921560: @_hamster_coin @cz_binance @...,rt @bufi79921560: @_hamster_coin @cz_binance @...
4,512353,RT @easyeight08: 50k NEXT WEEK OR I DELETE MY ...,rt @easyeight08: 50k next week or i delete my ...


In [15]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt    

In [16]:
#data['processed_tweet_new2'] = data['processed_tweet_new2'].str.replace("[^a-zA-Z#]", " ")
#Removing RT, Punctuation etc
remove_rt = lambda x: re.sub('rt @\w+: '," ",x)
rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(http\S+)"," ",x)
data_dd['processed_tweet_new3'] = data_dd.processed_tweet_new.map(remove_rt).map(rt)
data_dd['processed_tweet_new3'] = data_dd.processed_tweet_new3.str.lower()
#data_dd['processed_tweet_new2'] = np.vectorize(remove_pattern)(data_dd['processed_tweet_new'], "@[\w]*")

In [18]:
data_dd.head(15)

,unique_id,tweet,processed_tweet_new,processed_tweet_new3
0,480075,RT @OfficialTravlad: 🔥 SafeRiseV2 🔥 \n\n🦅 Risi...,rt @officialtravlad: 🔥 saferisev2 🔥 \n\n🦅 risi...,saferisev2 rising from the bot litter...
1,360524,Holly shit. I just found Mute functionality in...,holly shit. i just found mute functionality in...,holly shit i just found mute functionality in...
2,476916,RT @MiniDOGEToken: MiniDOGE is about to take o...,rt @minidogetoken: minidoge is about to take o...,minidoge is about to take over the crypto spa...
3,446465,RT @Bufi79921560: @_hamster_coin @cz_binance @...,rt @bufi79921560: @_hamster_coin @cz_binance @...,hamster coin binance news hotbit ...
4,512353,RT @easyeight08: 50k NEXT WEEK OR I DELETE MY ...,rt @easyeight08: 50k next week or i delete my ...,50k next week or i delete my twitter btc
5,494230,"RT @TronixTrx: Next bull run ❤️:\n#BTC : 100,0...","rt @tronixtrx: next bull run ❤️:\n#btc : 100,0...",next bull run btc 100 000 bnb 2 00...
6,480037,RT @OfficialTravlad: 💎@babyfegtoken🦍Feg Day Is...,rt @officialtravlad: 💎@babyfegtoken🦍feg day is...,feg day is coming join babyfeg and the t...
7,446490,RT @BullMission: We're all on a bull mission t...,rt @bullmission: we are all on a bull mission ...,we are all on a bull mission together btc ...
8,239795,@elonmusk $REN $TFUEL $MBL $DOGE $LET $EM $BTC...,@elonmusk $ren $tfuel $mbl $doge $let $em $btc...,ren tfuel mbl doge let em btc eth x...
9,528106,RT @rodneyprobst: @CrypticTycoon 0x392e0dfEe02...,rt @rodneyprobst: @cryptictycoon 0x392e0dfee02...,0x392e0dfee02271e6933f948caa1e4df4ee624148 ...


In [19]:
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from textblob import TextBlob

stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

tokenizer = RegexpTokenizer(r"\w+")

In [20]:
def preprocess(text):
    words = [lemmatizer.lemmatize(w) for w in text if w not in stop_words]
    stem_text = " ".join([stemmer.stem(i) for i in words])
    return stem_text

In [21]:
data_dd["tweet_new"] = data_dd["processed_tweet_new3"].apply(
    lambda x: preprocess(tokenizer.tokenize(x.lower()))
)

In [22]:
data_dd.head()

,unique_id,tweet,processed_tweet_new,processed_tweet_new3,tweet_new
0,480075,RT @OfficialTravlad: 🔥 SafeRiseV2 🔥 \n\n🦅 Risi...,rt @officialtravlad: 🔥 saferisev2 🔥 \n\n🦅 risi...,saferisev2 rising from the bot litter...,saferisev2 rise bot litter ash v1 new plan har...
1,360524,Holly shit. I just found Mute functionality in...,holly shit. i just found mute functionality in...,holly shit i just found mute functionality in...,holli shit found mute function twitter btc max...
2,476916,RT @MiniDOGEToken: MiniDOGE is about to take o...,rt @minidogetoken: minidoge is about to take o...,minidoge is about to take over the crypto spa...,minidog take crypto space featur first ever au...
3,446465,RT @Bufi79921560: @_hamster_coin @cz_binance @...,rt @bufi79921560: @_hamster_coin @cz_binance @...,hamster coin binance news hotbit ...,hamster coin binanc news hotbit hamarmi hamste...
4,512353,RT @easyeight08: 50k NEXT WEEK OR I DELETE MY ...,rt @easyeight08: 50k next week or i delete my ...,50k next week or i delete my twitter btc,50k next week delet twitter btc


In [23]:
data_dd[['polarity', 'subjectivity']] = data_dd['tweet_new'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

In [24]:
data_dd.head()
#data_dd.to_csv('data_cleaned.csv')

In [25]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt

def clean_tweets(tweets):
    #remove twitter Return handles (RT @xxx:)
    tweets = np.vectorize(remove_pattern)(tweets, "RT @[\w]*:") 
    
    #remove twitter handles (@xxx)
    tweets = np.vectorize(remove_pattern)(tweets, "@[\w]*")
    
    #remove URL links (httpxxx)
    tweets = np.vectorize(remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
    
    #remove special characters, numbers, punctuations (except for #)
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
    
    return tweets

data_dd['tweet_new2'] = clean_tweets(data_dd['tweet'])
data_dd['tweet_new2'].head()

0     🔥 SafeRiseV2 🔥 \n\n🦅 Rising from the bot-litt...
1    Holly shit. I just found Mute functionality in...
2     MiniDOGE is about to take over the crypto spa...
3           #hamarmy\n#Hamsterarmy\n\nContract addres…
4           50k NEXT WEEK OR I DELETE MY TWITTER! $BTC
Name: tweet_new2, dtype: object

In [21]:
data_dd.head(30)

,unique_id,tweet,processed_tweet_new,processed_tweet_new3,tweet_new,polarity,subjectivity,tweet_new2
0,396707,RT @JohnnyInCrypto: Good morning future #Crypt...,rt @johnnyincrypto: good morning future #crypt...,good morning future crypto millionaires mar...,good morn futur crypto millionair market look ...,0.700000,0.600000,Good morning future #Crypto millionaires! Mar...
1,125695,@CrytpoI @hicetnunc2000 fortunately for me i'm...,@crytpoi @hicetnunc2000 fortunately for me i a...,fortunately for me i am not in a position ...,fortun posit receiv repercuss whatsoev word ch...,0.525000,0.500000,fortunately for me i'm not in a position whe...
2,435284,RT @glitchytoken: WE ARE LIVE!\n\nYou can now ...,rt @glitchytoken: we are live!\n\nyou can now ...,we are live you can now trade gly on pancak...,live trade gli pancakeswap slippag 17 glitchi ...,0.136364,0.500000,WE ARE LIVE!\n\nYou can now trade GLY on Panc...
3,569790,🟩🟩🟩🟩🟩🟩🟩🟩🟩\n🟩⬜️⬜️⬜️🟩⬜️⬜️⬜️🟩\n🟩⬜️⬛️⬛️🟩⬜️⬛️⬛️🟩\n🟩...,🟩🟩🟩🟩🟩🟩🟩🟩🟩\n🟩⬜️⬜️⬜️🟩⬜️⬜️⬜️🟩\n🟩⬜️⬛️⬛️🟩⬜️⬛️⬛️🟩\n🟩...,...,unidex aggreg aggreg avail multipl chain eth m...,0.000000,0.000000,🟩🟩🟩🟩🟩🟩🟩🟩🟩\n🟩⬜️⬜️⬜️🟩⬜️⬜️⬜️🟩\n🟩⬜️⬛️⬛️🟩⬜️⬛️⬛️🟩\n🟩...
4,320991,I just joined the #ETHPAD #WHITELIST EVENT. he...,i just joined the #ethpad #whitelist event. he...,i just joined the ethpad whitelist event he...,join ethpad whitelist event team behind record...,0.216667,0.716667,I just joined the #ETHPAD #WHITELIST EVENT. he...
5,360634,"Omg, this expert had made a forecast that bitc...","omg, this expert had made a forecast that bitc...",omg this expert had made a forecast that bitc...,omg expert made forecast bitcoin cost fell 64k...,0.175000,0.325000,"Omg, this expert had made a forecast that bitc..."
6,402437,RT @Mia_gives: $1500 in $ETH to a random perso...,rt @mia_gives: $1500 in $eth to a random perso...,1500 in eth to a random person who follows ...,1500 eth random person follow retweet tag thre...,-0.500000,0.500000,"$1500 in $ETH to a random person who follows,..."
7,375911,RT @BTCTN: The 3 #Smartbch nodes are operated ...,rt @btctn: the 3 #smartbch nodes are operated ...,the 3 smartbch nodes are operated by viabtc ...,3 smartbch node oper viabtc matrixport smartbc...,0.250000,0.333333,"The 3 #Smartbch nodes are operated by Viabtc,..."
8,388256,RT @ElisaCrypto: $1000 in $ETH to a random per...,rt @elisacrypto: $1000 in $eth to a random per...,1000 in eth to a random person who follows ...,1000 eth random person follow retweet ethereum,-0.500000,0.500000,$1000 in $ETH to a random person who follows ...
9,372802,RT @AirdropStario: 💧Pet Bet Airdrop 💧\n\n🏆 Tas...,rt @airdropstario: 💧pet bet airdrop 💧\n\n🏆 tas...,pet bet airdrop task 200 ...,pet bet airdrop task 200 billion pbet referr 4...,0.000000,0.000000,💧Pet Bet Airdrop 💧\n\n🏆 Task: ➕ 20...


In [26]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
row=data_dd['tweet_new2'][22]
score = SentimentIntensityAnalyzer().polarity_scores(row)
print(score)

{'neg': 0.0, 'neu': 0.792, 'pos': 0.208, 'compound': 0.6369}


In [27]:
for index, row in data_dd['tweet_new2'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    #score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    data_dd.loc[index, 'neg'] = neg
    data_dd.loc[index, 'neu'] = neu
    data_dd.loc[index, 'pos'] = pos
    data_dd.loc[index, 'compound'] = comp

In [28]:
#output the score file
data_dd.to_csv('data_btc_july_sentiment2.csv')

### Merge Tweet data and Sentiment Scores

In [18]:
#Input the sentiment file and merge with original data
data_dd_sent = pd.read_csv('data_btc_july_sentiment2.csv')

In [19]:
data_dd_sent.head(10)
#data.head(10)

,Unnamed: 0,unique_id,tweet,processed_tweet_new,processed_tweet_new3,tweet_new,polarity,subjectivity,tweet_new2,neg,neu,pos,compound
0,0,480075,RT @OfficialTravlad: 🔥 SafeRiseV2 🔥 \n\n🦅 Risi...,rt @officialtravlad: 🔥 saferisev2 🔥 \n\n🦅 risi...,saferisev2 rising from the bot litter...,saferisev2 rise bot litter ash v1 new plan har...,-0.077652,0.498106,🔥 SafeRiseV2 🔥 \n\n🦅 Rising from the bot-litt...,0.059,0.551,0.390,0.8316
1,1,360524,Holly shit. I just found Mute functionality in...,holly shit. i just found mute functionality in...,holly shit i just found mute functionality in...,holli shit found mute function twitter btc max...,-0.200000,0.800000,Holly shit. I just found Mute functionality in...,0.231,0.769,0.000,-0.5574
2,2,476916,RT @MiniDOGEToken: MiniDOGE is about to take o...,rt @minidogetoken: minidoge is about to take o...,minidoge is about to take over the crypto spa...,minidog take crypto space featur first ever au...,0.250000,0.333333,MiniDOGE is about to take over the crypto spa...,0.000,1.000,0.000,0.0000
3,3,446465,RT @Bufi79921560: @_hamster_coin @cz_binance @...,rt @bufi79921560: @_hamster_coin @cz_binance @...,hamster coin binance news hotbit ...,hamster coin binanc news hotbit hamarmi hamste...,0.000000,0.000000,#hamarmy\n#Hamsterarmy\n\nContract addres…,0.000,1.000,0.000,0.0000
4,4,512353,RT @easyeight08: 50k NEXT WEEK OR I DELETE MY ...,rt @easyeight08: 50k next week or i delete my ...,50k next week or i delete my twitter btc,50k next week delet twitter btc,0.000000,0.000000,50k NEXT WEEK OR I DELETE MY TWITTER! $BTC,0.000,1.000,0.000,0.0000
5,5,494230,"RT @TronixTrx: Next bull run ❤️:\n#BTC : 100,0...","rt @tronixtrx: next bull run ❤️:\n#btc : 100,0...",next bull run btc 100 000 bnb 2 00...,next bull run btc 100 000 bnb 2 000 trx 12 9 b...,0.000000,0.000000,"Next bull run ❤️:\n#BTC : 100,000$\n#BNB : 2,...",0.000,0.901,0.099,0.2960
6,6,480037,RT @OfficialTravlad: 💎@babyfegtoken🦍Feg Day Is...,rt @officialtravlad: 💎@babyfegtoken🦍feg day is...,feg day is coming join babyfeg and the t...,feg day come join babyfeg team next generat bu...,0.000000,0.000000,💎🦍Feg Day Is Coming 🦍 Join BABYFEG And The Te...,0.066,0.800,0.134,0.3510
7,7,446490,RT @BullMission: We're all on a bull mission t...,rt @bullmission: we are all on a bull mission ...,we are all on a bull mission together btc ...,bull mission togeth btc eth alt,0.000000,0.000000,We're all on a bull mission together! $BTC $E...,0.000,1.000,0.000,0.0000
8,8,239795,@elonmusk $REN $TFUEL $MBL $DOGE $LET $EM $BTC...,@elonmusk $ren $tfuel $mbl $doge $let $em $btc...,ren tfuel mbl doge let em btc eth x...,ren tfuel mbl doge let em btc eth xrp,0.000000,0.000000,$REN $TFUEL $MBL $DOGE $LET $EM $BTC $ETH $XR...,0.000,1.000,0.000,0.0000
9,9,528106,RT @rodneyprobst: @CrypticTycoon 0x392e0dfEe02...,rt @rodneyprobst: @cryptictycoon 0x392e0dfee02...,0x392e0dfee02271e6933f948caa1e4df4ee624148 ...,0x392e0dfee02271e6933f948caa1e4df4ee624148 bab...,0.000000,0.000000,0x392e0dfEe02271E6933F948CAa1E4df4eE624148 #...,0.000,1.000,0.000,0.0000


Create flag for different sentiment based on the compound score

In [20]:
def flag_df(df):

    if (df['neg'] > df['pos']):
        return 'negative'
    elif (df['neg'] < df['pos']):
        return 'positive'
    else:
        return 'neutral'

def flag_df1(df):

    if (df['compound'] <= - 0.05):
        return 'negative'
    elif (df['compound'] >= 0.05):
        return 'positive'
    else:
        return 'neutral'

data_dd_sent['sentiment']= data_dd_sent.apply(flag_df1, axis = 1)

In [21]:
data_dd_sent.head()

,Unnamed: 0,unique_id,tweet,processed_tweet_new,processed_tweet_new3,tweet_new,polarity,subjectivity,tweet_new2,neg,neu,pos,compound,sentiment
0,0,480075,RT @OfficialTravlad: 🔥 SafeRiseV2 🔥 \n\n🦅 Risi...,rt @officialtravlad: 🔥 saferisev2 🔥 \n\n🦅 risi...,saferisev2 rising from the bot litter...,saferisev2 rise bot litter ash v1 new plan har...,-0.077652,0.498106,🔥 SafeRiseV2 🔥 \n\n🦅 Rising from the bot-litt...,0.059,0.551,0.39,0.8316,positive
1,1,360524,Holly shit. I just found Mute functionality in...,holly shit. i just found mute functionality in...,holly shit i just found mute functionality in...,holli shit found mute function twitter btc max...,-0.200000,0.800000,Holly shit. I just found Mute functionality in...,0.231,0.769,0.00,-0.5574,negative
2,2,476916,RT @MiniDOGEToken: MiniDOGE is about to take o...,rt @minidogetoken: minidoge is about to take o...,minidoge is about to take over the crypto spa...,minidog take crypto space featur first ever au...,0.250000,0.333333,MiniDOGE is about to take over the crypto spa...,0.000,1.000,0.00,0.0000,neutral
3,3,446465,RT @Bufi79921560: @_hamster_coin @cz_binance @...,rt @bufi79921560: @_hamster_coin @cz_binance @...,hamster coin binance news hotbit ...,hamster coin binanc news hotbit hamarmi hamste...,0.000000,0.000000,#hamarmy\n#Hamsterarmy\n\nContract addres…,0.000,1.000,0.00,0.0000,neutral
4,4,512353,RT @easyeight08: 50k NEXT WEEK OR I DELETE MY ...,rt @easyeight08: 50k next week or i delete my ...,50k next week or i delete my twitter btc,50k next week delet twitter btc,0.000000,0.000000,50k NEXT WEEK OR I DELETE MY TWITTER! $BTC,0.000,1.000,0.00,0.0000,neutral


In [22]:
#Merge data with original tweet data
merg2 = merge1.merge(data_dd_sent[['unique_id','tweet_new','tweet_new2','polarity','subjectivity','neg','neu','pos','compound','sentiment']],on='unique_id',how='left' )

In [23]:
merg2.columns

Index(['author id', 'created_at_x', 'geo', 'id', 'lang', 'like_count',
       'quote_count', 'reply_count', 'retweet_count', 'source', 'tweet',
       'referenced_tweets_id', 'referenced_tweets_type', 'media', 'unique_id',
       'author_id', 'created_at_y', 'location', 'verified', 'username',
       'followers_count', 'following_count', 'listed_count', 'tweet_count',
       'name', 'description', 'media_key', 'media_type', 'media_url', 'width',
       'height', 'duration_ms', 'view_count', 'preview_image_url', 'tweet_new',
       'tweet_new2', 'polarity', 'subjectivity', 'neg', 'neu', 'pos',
       'compound', 'sentiment'],
      dtype='object')

In [43]:
col_list = ['author id', 'created_at_x', 'geo', 'id', 'lang', 'like_count',
       'quote_count', 'reply_count', 'retweet_count', 'source', 'tweet',
       'referenced_tweets_id', 'referenced_tweets_type', 'media', 'unique_id',
        'verified', 'username','followers_count', 'following_count', 'listed_count', 'tweet_count',
       'name', 'description',  'media_type', 'media_url', 
      'view_count',  'tweet_new',
       'tweet_new2', 'polarity', 'subjectivity', 'neg', 'neu', 'pos',
       'compound', 'sentiment']
final_data_senti = merg2[col_list]

In [50]:
final_data_senti.head(50)

,author id,created_at_x,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,...,tweet_new,tweet_new2,polarity,subjectivity,neg,neu,pos,compound,sentiment,created_date
0,2753291192,2021-07-01 18:47:32+00:00,,1410671428450541568,en,0,0,0,110,Twitter for Android,...,saferisev2 rise bot litter ash v1 new plan har...,🔥 SafeRiseV2 🔥 \n\n🦅 Rising from the bot-litt...,-0.077652,0.498106,0.059,0.551,0.390,0.8316,positive,2021-07-01 18:00:00+00:00
1,150624586,2021-07-01 18:47:31+00:00,,1410671426495991812,en,1,0,0,0,Twitter for Android,...,holli shit found mute function twitter btc max...,Holly shit. I just found Mute functionality in...,-0.200000,0.800000,0.231,0.769,0.000,-0.5574,negative,2021-07-01 18:00:00+00:00
2,1400283959041224706,2021-07-01 18:47:31+00:00,,1410671426265174025,en,0,0,0,8687,Twitter for Android,...,minidog take crypto space featur first ever au...,MiniDOGE is about to take over the crypto spa...,0.250000,0.333333,0.000,1.000,0.000,0.0000,neutral,2021-07-01 18:00:00+00:00
3,952526506051371008,2021-07-01 18:47:31+00:00,,1410671425401372673,en,0,0,0,2,Twitter for Android,...,hamster coin binanc news hotbit hamarmi hamste...,#hamarmy\n#Hamsterarmy\n\nContract addres…,0.000000,0.000000,0.000,1.000,0.000,0.0000,neutral,2021-07-01 18:00:00+00:00
4,1371937887109980161,2021-07-01 18:47:29+00:00,,1410671418652643334,en,0,0,0,194,Twitter for Android,...,50k next week delet twitter btc,50k NEXT WEEK OR I DELETE MY TWITTER! $BTC,0.000000,0.000000,0.000,1.000,0.000,0.0000,neutral,2021-07-01 18:00:00+00:00
5,3628204632,2021-07-01 18:47:29+00:00,,1410671417750790148,en,0,0,0,299,Twitter Web App,...,next bull run btc 100 000 bnb 2 000 trx 12 9 b...,"Next bull run ❤️:\n#BTC : 100,000$\n#BNB : 2,...",0.000000,0.000000,0.000,0.901,0.099,0.2960,positive,2021-07-01 18:00:00+00:00
6,712941404175998976,2021-07-01 18:47:29+00:00,,1410671417729765376,en,0,0,0,545,Twitter for Android,...,feg day come join babyfeg team next generat bu...,💎🦍Feg Day Is Coming 🦍 Join BABYFEG And The Te...,0.000000,0.000000,0.066,0.800,0.134,0.3510,positive,2021-07-01 18:00:00+00:00
7,2163163850,2021-07-01 18:47:29+00:00,,1410671417591468037,en,0,0,0,5977,Twitter for Android,...,bull mission togeth btc eth alt,We're all on a bull mission together! $BTC $E...,0.000000,0.000000,0.000,1.000,0.000,0.0000,neutral,2021-07-01 18:00:00+00:00
8,436410772,2021-07-01 18:47:29+00:00,,1410671416341532677,en,1,0,1,0,Twitter for Android,...,ren tfuel mbl doge let em btc eth xrp,$REN $TFUEL $MBL $DOGE $LET $EM $BTC $ETH $XR...,0.000000,0.000000,0.000,1.000,0.000,0.0000,neutral,2021-07-01 18:00:00+00:00
9,1147769435618287617,2021-07-01 18:47:28+00:00,,1410671412499468291,pt,0,0,0,1,Twitter,...,0x392e0dfee02271e6933f948caa1e4df4ee624148 bab...,0x392e0dfEe02271E6933F948CAa1E4df4eE624148 #...,0.000000,0.000000,0.000,1.000,0.000,0.0000,neutral,2021-07-01 18:00:00+00:00


In [49]:
import datetime
final_data_senti = final_data_senti.assign(created_date=pd.to_datetime(final_data_senti['created_at_x']).dt.floor('H'))

In [ ]:
##save file in a csv to create input features in the next set of data preparation
final_data_senti.to_csv('final_data_senti_aug15.csv')